In [13]:
import soundfile as sf
import numpy as np

noisy_sensor_tuple = sf.read('resources/noisySensor.wav')

In [14]:
sampling_rate = noisy_sensor_tuple[1]

In [15]:
noisy_sensor_data = noisy_sensor_tuple[0]
noisy_sensor_data

array([[ 0.00793457,  0.03259277, -0.00076294,  0.0027771 ],
       [ 0.00717163, -0.01785278,  0.00039673,  0.0300293 ],
       [ 0.00408936, -0.00509644, -0.00576782, -0.00244141],
       ...,
       [-0.01037598,  0.00057983, -0.02798462, -0.01687622],
       [ 0.00997925, -0.00830078,  0.00979614, -0.02633667],
       [-0.02468872, -0.01419067,  0.00164795, -0.00024414]])

In [63]:
noisy_sensor_data.shape

(109466, 4)

In [7]:
frame_length = 128
frame_shift = 32
frame_length_samples = frame_length * sampling_rate / 1000

In [8]:
def my_windowing(v_signal: np.ndarray, sampling_rate: int, frame_length: int, frame_shift: int) -> [np.ndarray, np.ndarray]:
    millis_per_sample = 1000 / sampling_rate
    frame_length_num_points = frame_length / millis_per_sample
    frame_shift_num_points = frame_shift / millis_per_sample
    num_frames = int(np.floor((len(v_signal) - frame_length_num_points) / frame_shift_num_points) + 1)
    m_frames = np.zeros((num_frames, int(frame_length_num_points)))
    v_time_frame = np.zeros(num_frames)
    for i in range(num_frames):
        start = int(i * frame_shift_num_points)
        end = int(i * frame_shift_num_points + frame_length_num_points)
        m_frames[i] = v_signal[start:end]
        v_time_frame[i] = (start + end)/(2*sampling_rate)
    return [m_frames, v_time_frame]

In [9]:
def compute_freq_axis(m_stft: np.ndarray):
    return np.linspace(0, sampling_rate//2, num=m_stft.shape[1])

def remove_upper_half_spectrum(m_stft: np.ndarray) -> np.ndarray:
    m_stft_new = m_stft[:, :(int(m_stft.shape[1]/2)+1)]
    return m_stft_new

In [10]:
def compute_stft(v_signal: np.ndarray, fs: int, frame_length: int, frame_shift: int, v_analysis_window: np.ndarray) -> [np.ndarray, np.ndarray, np.ndarray]:
    m_frames, v_time_frame = my_windowing(v_signal, fs, frame_length, frame_shift)
    m_stft_full = np.zeros(m_frames.shape, dtype=np.complex128)
    #v_analysis_window = v_analysis_window(m_frames.shape[1])
    for i in range(m_frames.shape[0]):
        m_stft_full[i] = np.fft.fft(m_frames[i]*v_analysis_window)
    #v_freq = np.fft.rfftfreq(m_stft_full.shape[1], 1/fs)
    m_stft = remove_upper_half_spectrum(m_stft_full)
    v_freq = compute_freq_axis(m_stft)
    return [m_stft, v_freq, v_time_frame]

In [35]:
from scipy.signal import get_window

hann_window = get_window('hann', int(frame_length_samples), fftbins=True)
root_hann = np.sqrt(hann_window)
analysis_window = root_hann
synthesis_window = root_hann*0.5

In [36]:
noisy_sensor_data[:, 1]

array([ 0.03259277, -0.01785278, -0.00509644, ...,  0.00057983,
       -0.00830078, -0.01419067])

In [37]:
m_stft_all, v_freq_all, v_time_frame_all = list(), list(), list()

for i in range(noisy_sensor_data.shape[1]):
    m_stft, v_freq, v_time_frame = compute_stft(noisy_sensor_data[:, i], sampling_rate, frame_length, frame_shift, analysis_window)
    m_stft_all.append(m_stft)
    v_freq_all.append(v_freq)
    v_time_frame_all.append(v_time_frame)

m_stft_all = np.array(m_stft_all)
v_freq_all = np.array(v_freq_all)
v_time_frame_all = np.array(v_time_frame_all)

2.2) The synthesis window is multiplied by one half to compensate for the overlap of the windows. The synthesis window is then applied to the STFT of the noisy signal to obtain the reconstructed signal.

In [42]:
def compute_time_delay(d, i, theta):
    c = 340
    t = i*d*np.cos(theta) / c
    return t

In [43]:
theta = np.pi/4
d = 0.05
ts = np.array([compute_time_delay(d, i, theta) for i in range(noisy_sensor_data.shape[1])])
ts

array([0.        , 0.00010399, 0.00020797, 0.00031196])

In [45]:
v_freq_all.shape[1]

1025

It is the correct formula because there is a right angle can be projected between two sensors and then we can calculate the time difference using triangulation.

In [48]:
def compute_steering_vector(sampling_rate, t):
    N = v_freq_all.shape[1]
    a = np.zeros((N, 4), dtype=np.complex128)
    for k in range(N):
        a[k][0] = 1
        for i in range(1, 4):
            a[k][i] = np.exp((-1j*2*np.pi*k*sampling_rate*t[i])/N)
    return a

In [60]:
a = compute_steering_vector(sampling_rate, ts)
a.shape

(1025, 4)

In [61]:
m = noisy_sensor_data.shape[1]
ah = np.conjugate(a).T
ah.shape

(4, 1025)

In [62]:
m_stft_all.shape

(4, 210, 1025)

In [73]:
b = m_stft_all.reshape(210, 4, 1025)
b.shape

(210, 4, 1025)

In [72]:
s = np.zeros((b.shape[0], b.shape[2]), dtype=np.complex128)
s.shape

(210, 1025)

In [77]:
b[0].shape

(4, 1025)

In [84]:
for i in range(b.shape[0]):
    s[i] = ah @ b[i]
s.shape

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 4 is different from 1025)